# ETL LegalRES

* Extract: Bajar los JSON de legalRES de S3 
* Transform: Seleccionar las columnas que interesan, aplanar a una tabla y generar columnas nuevas
* Load: Cargar tabla limpia a S3

In [1]:
#!pip install awswrangler
#!pip install --upgrade pandas

In [2]:
import pandas as pd
from sagemaker import get_execution_role
import boto3
from tqdm.notebook import tqdm

import awswrangler as wr

import json
from pandas.io.json import json_normalize

#data = pd.read_json(data_location)

ModuleNotFoundError: No module named 'sagemaker'

In [ ]:
bucket='legalres-storage-study-json'
data_key = 'AC000lvffws1.json'
data_location = 's3://{}/{}'.format(bucket, data_key)


s3 = boto3.client('s3')


Config datos ya extraidos para no duplicar

In [4]:
prefix_processed = 'processed'
data_location_processed = 's3://{}/{}/'.format(bucket, prefix_processed)

## Extract 

Datos ya extraidos para no duplicar

In [5]:
df_processed = wr.s3.read_parquet(path=data_location_processed,
                                  columns=['cve'])

537735

798060

880520

881320

In [6]:
df_processed.shape

(881320, 1)

In [7]:
cve_processed = list(df_processed['cve'].values)

In [8]:
assert len(cve_processed) == len(df_processed.index)

In [9]:
'AC0AEHf1alMr' in cve_processed

True

Prueba con un archivo para ver estructura

In [10]:
data_stream = s3.get_object(Bucket=bucket,Key=data_key)['Body'].read().decode('utf-8')
#data = json.loads(data_stream)
#print(json.dumps(data, indent=4, ensure_ascii=False))

In [11]:
"\"tipo_actuacion\": \"CONSTITUCION\"" in data_stream[:200]

True

In [12]:
data_stream = s3.get_object(Bucket=bucket,Key="AC0Ani2IvrvE.json", Range='bytes=0-300')['Body'].read().decode('utf-8')
print(data_stream)
#data = json.loads(data_stream)
#print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "error_fatal": false,
    "disuelto": true,
    "rut": "77181329-1",
    "fecha": "2021-06-02",
    "cve": "AC0Ani2IvrvE",
    "tipo_actuacion": "ANOTACION",
    "tipo_sociedad": "NA",
    "estructura": [
        {
            "numero": 1,
            "tipo": "titulo_primero",
            "text


In [13]:
'ANOTACION' in data_stream

True

Helper functions

In [14]:
def find_item(struct, field='tipo', string=''):
    for item in struct:
        if item[field] == string:
            return item
    return None

In [15]:
def get_data_study(dj):
    """
    Parses the relevant data from a JSON
    
    inputs
    ------
    
    dj: json 
        data jason
        
    
    outputs
    -------
    
    dict
        With the relevant columns
    """
    
    parsed = dict()
    
    parsed['rut'] = dj['rut']
    parsed['cve'] = dj['cve']
    parsed['fecha'] = dj['fecha']
    parsed['actuacion'] = dj['tipo_actuacion']
    parsed['lugar'] = find_item(dj['estructura'][1]['articulos'],
                                 field='articulo',
                                 string='inicio')['texto']
    parsed['tipo_sociedad'] = dj['tipo_sociedad']
    
    parsed['objeto'] = find_item(dj['estructura'][1]['articulos'],
                                 field='subtitulo',
                                 string='OBJETO')['texto']
    
    parsed['firmas'] = find_item(dj['estructura'], 
                                 field='tipo',
                                 string='firmas')['firmas']
    
    return parsed
    

Test helper

Objetos para recorrer carpeta con JSON

In [16]:
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket)

El loop principal que recorre todos los archivos

**Tarda 12 horas en procesar toda la carpeta**

In [66]:
class file_utilities:
    """file handling function"""

    def get_keys_from_prefix(self, bucket):
        '''gets list of keys and dates for given bucket and prefix'''
        keys_list = []
        paginator = s3.get_paginator('list_objects_v2')
        # use Delimiter to limit search to that level of hierarchy
        for page in tqdm(paginator.paginate(Bucket=bucket), total=1200):
            keys = [content['Key'] for content in page.get('Contents')]
            # print('keys in page: ', len(keys))
            keys_list.extend(keys)
        return keys_list
    
    def process_study_json(self, bucket, key):
        """read json file"""
        data_stream = s3.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
        progress_bar.update(1)
        if "CONSTITUCION" in data_stream[:200]:
            data = json.loads(data_stream)
            dictout = get_data_study(data)
            return dictout

        return None

In [67]:
fu = file_utilities()

In [18]:

all_keys = fu.get_keys_from_prefix(bucket)

In [19]:
#keys_not_processed = [key for key in all_keys if key.split('.')[0] not in cve_processed]

In [46]:
all_keys_set = set([key.split('.')[0] for key in all_keys if 'processed' not in key])

In [47]:
cve_processed_set = set(cve_processed)

In [48]:
keys_not_processed = all_keys_set.difference(cve_processed_set)

In [49]:
keys_not_processed = [str(key) + '.json' for key in keys_not_processed]

In [50]:
len(keys_not_processed)

222502

In [51]:
proc = [x for x in keys_not_processed if 'processed' in x]

In [53]:
import os
from multiprocessing import Pool
from itertools import repeat
import pandas as pd
import json

In [68]:
def get_dflist_multiprocess(keys_list, progress_bar, num_proc=4):
    with Pool(num_proc) as pool:
        df_list = pool.starmap(fu.process_study_json, zip(repeat(bucket), keys_list), 15)
        pool.close()
        pool.join()
    return df_list

In [69]:
num_proc = os.cpu_count() #number of processors
print(num_proc)

2


In [77]:
progress_bar = tqdm(total=len(keys_not_processed[:1000]))
df_list = get_dflist_multiprocess(keys_not_processed[:1000], progress_bar, num_proc=num_proc) #collect dataframes for each file


In [78]:
[x for x in df_list if x is not None]

[]

In [ ]:

LIMIT = 1 # early stopping for debugging purposes
i = 0 # pages
j = 0 # constituciones

output = []
failed_cve = []

for page in tqdm(pages, total=1200):
    i = i + 1
    contents = page['Contents']

    keys = [obj['Key'] for obj in contents]
    #keys_not_processed = [key for key in keys if key.split('.')[0] not in cve_processed]
    keys_not_processed = set([key.split('.')[0] for key in keys]).difference(set(cve_processed))
    keys_not_processed = [str(s) + '.json' for s in keys_not_processed if 'processed' not in s]

    j = 0
    k = 0
    for key in keys_not_processed:
        #key = obj['Key'""]
        #key_cve = key.split('.')
        #if key_cve not in cve_processed:

        # Read data
        data_stream = s3.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')

        try:
            #if data['tipo_actuacion'] == 'CONSTITUCION':
            if "CONSTITUCION" in data_stream[:200]:
                data = json.loads(data_stream)
                dictout = get_data_study(data)
                output.append(dictout)
                j = j + 1
            elif "MODIFICACION" in data_stream[:200]:
                k = k + 1
            elif "TRANSFORMACION" in data_stream[:200]:
                k = k + 1
            elif "DISOLUCION" in data_stream[:200]:
                k = k + 1
            elif "EMIGRACION" in data_stream[:200]:
                k = k + 1
            elif "MIGRACION" in data_stream[:200]:
                k = k + 1
            elif "SANEAMIENTO" in data_stream[:200]:
                k = k + 1
            elif "RECTIFICACION" in data_stream[:200]:
                k = k + 1
            elif "ANOTACION" in data_stream[:200]:
                k = k + 1

            else:
                print(key)
        except:
            failed_cve.append(key)

    print(i, '/ New keys in page:', len(keys_not_processed), '| Sample', keys_not_processed[0], 
          '| Processed:', j, '| Other:', k, '| Total Failed:', len(failed_cve))

    #if i >= LIMIT:
    #    break

1 / New keys in page: 200 | Sample AC00uoenQA5k.json | Processed: 0 | Other: 200 | Total Failed: 0
2 / New keys in page: 216 | Sample AC02SbPpwjtD.json | Processed: 0 | Other: 216 | Total Failed: 0
3 / New keys in page: 177 | Sample AC06MVpDajef.json | Processed: 0 | Other: 177 | Total Failed: 0
4 / New keys in page: 186 | Sample AC0CQEJ1pbFA.json | Processed: 0 | Other: 186 | Total Failed: 0
5 / New keys in page: 201 | Sample AC0DA1yUCD8f.json | Processed: 0 | Other: 201 | Total Failed: 0
6 / New keys in page: 190 | Sample AC0IU2UiXxcF.json | Processed: 0 | Other: 190 | Total Failed: 0
7 / New keys in page: 204 | Sample AC0MJqLsSMXy.json | Processed: 0 | Other: 204 | Total Failed: 0
8 / New keys in page: 191 | Sample AC0QsOSnomDX.json | Processed: 0 | Other: 191 | Total Failed: 0
9 / New keys in page: 196 | Sample AC0TzUEYc1mg.json | Processed: 0 | Other: 196 | Total Failed: 0
10 / New keys in page: 208 | Sample AC0VLPJ1qD6l.json | Processed: 0 | Other: 208 | Total Failed: 0
11 / New 

In [80]:
#!pip install line_profiler

In [81]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [82]:
%lprun -f get_actuaciones get_actuaciones()

Timer unit: 1e-06 s

Total time: 21.9568 s
File: <ipython-input-79-b5e97e399272>
Function: get_actuaciones at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_actuaciones():
     2                                           
     3         1          2.0      2.0      0.0      LIMIT = 1 # early stopping for debugging purposes
     4         1          1.0      1.0      0.0      i = 0 # pages
     5         1          1.0      1.0      0.0      j = 0 # constituciones
     6                                           
     7         1          1.0      1.0      0.0      output = []
     8         1          1.0      1.0      0.0      failed_cve = []
     9                                           
    10         1     853736.0 853736.0      3.9      for page in tqdm(pages, total=1200):
    11         1          2.0      2.0      0.0          i = i + 1
    12         1          1.0      1.0      0.0          con

1 / New keys in page: 196 | Sample AC0RbK40QypB.json | Processed: 0 | Other: 196 | Total Failed: 0


In [45]:
df = pd.DataFrame(output)
df.head()

""


In [46]:
df.shape

(0, 0)

In [17]:
df.dtypes

rut              object
cve              object
fecha            object
lugar            object
tipo_sociedad    object
objeto           object
firmas           object
dtype: object

## Transform 

Campo fecha

In [18]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d")

In [19]:
df['year'] = df['fecha'].dt.year
df['month'] = df['fecha'].dt.month

In [20]:
df.head()

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas,year,month
0,77051535-1,AC000lvffws1,2019-08-06,"En MELIPEUCO, Región de LA ARAUCANIA, Chile, a...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'rut': '19.479.301-4', 'nombre': 'DAVID NICO...",2019,8
1,76707638-K,AC000rmdak0p,2017-02-24,"En CONCEPCION, Región del BIOBIO, Chile, a 24 ...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'rut': '12.006.720-6', 'nombre': 'MIGUEL ALE...",2017,2
2,77556566-7,AC0016ZGLrAC,2022-04-11,"En VALDIVIA, Región de LOS RIOS, Chile, a 11 d...",SOCIEDAD POR ACCIONES,"El objeto de la sociedad será a) Inversión, Ex...","[{'rut': '11.933.089-0', 'nombre': 'ROSA HERMI...",2022,4
3,77215315-5,AC001EmedaGk,2020-09-02,"En ÑUÑOA, Región METROPOLITANA DE SANTIAGO, Ch...",SOCIEDAD POR ACCIONES,El objeto de la sociedad será Consultoria en T...,"[{'rut': '17.118.202-6', 'nombre': 'DANTE FRAN...",2020,9
4,77462715-4,AC001EoYD4Yq,2021-10-14,"En PROVIDENCIA, Región METROPOLITANA DE SANTIA...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'rut': '18.848.785-8', 'nombre': 'LUIS FELIP...",2021,10


## Load 

En formato Apache Parquet

In [21]:


wr.s3.to_parquet(
    df,
    path='s3://{}/processed'.format(bucket),
    dataset=True,
    partition_cols=['year', 'month'],
    mode='append'
)

{'paths': ['s3://legalres-storage-study-json/processed/year=2013/month=5/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=7/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=8/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=10/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=11/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=12/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=1/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=2/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=3/275de328138f